# 🧠 AI Mind Map Generator

This notebook uses **Ollama** to generate a structured mind map from any text input and visualizes it using **Graphviz**.

### 🚀 Workflow
1.  **Input**: Provide text or a topic.
2.  **Generate**: AI extracts key concepts and relationships in Graphviz DOT format.
3.  **Visualize**: Render the mind map as an image.

In [16]:
# 📦 Install Dependencies
# Ensure graphviz is installed both as a Python package and a system binary.
# If you don't have the system binary, this might fail to render images.
!pip install graphviz requests


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# ⚙️ Configuration
import requests
import graphviz
from IPython.display import display
import shutil

OLLAMA_BASE_URL = 'http://localhost:11434'
OLLAMA_MODEL = 'phi3:mini' # Use a model capable of following formatting instructions

print(f"✅ Configured to use {OLLAMA_MODEL} at {OLLAMA_BASE_URL}")

✅ Configured to use phi3:mini at http://localhost:11434


In [18]:
# 📝 Input Text
# Replace this with your own text or document content
content = """
Artificial Intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. 
AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry. 
Machine learning feeds a computer data and uses statistical techniques to help it "learn" how to get progressively better at a task, without having been specifically programmed for that task, eliminating the need for millions of lines of written code. 
Deep learning is a type of machine learning that runs inputs through a biologically-inspired neural network architecture. The neural networks contain a number of hidden layers through which the data is processed, allowing the machine to go "deep" in its learning, making connections and weighting input for the best results.
"""
print(f"📄 Input text length: {len(content)} characters")

📄 Input text length: 959 characters


In [19]:
# 🧠 Generation Function
def generate_mindmap_dot(text):
    prompt = f"""
    You are an expert at creating mind maps. 
    Generate a valid Graphviz DOT source code to represent a mind map for the provided text.
    
    Rules:
    1. Use 'digraph MindMap {{ ... }}' structure.
    2. Use meaningful labels for nodes.
    3. Keep the structure hierarchical.
    4. Do NOT include any markdown formatting (like ```dot or ```), just return the raw DOT code.
    5. Style the nodes to look professional (e.g., shape=box, style=filled, fillcolor=lightblue).
    
    Text to map:
    {text}
    """
    
    payload = {
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": 0.3} # Low temperature for consistent formatting
    }
    
    try:
        print("⏳ Generatng mind map structure...")
        response = requests.post(f"{OLLAMA_BASE_URL}/api/generate", json=payload)
        if response.status_code == 200:
            dot_code = response.json()['response']
            # Clean up potential markdown code blocks if the model ignores the instruction
            dot_code = dot_code.replace("```dot", "").replace("```", "").strip()
            return dot_code
        else:
            print(f"❌ Error: {response.text}")
            return None
    except Exception as e:
        print(f"❌ Connection Failed: {e}")
        return None

In [20]:
# 🎨 Generate & Visualize
def check_graphviz():
    """Check if Graphviz 'dot' executable is available."""
    if shutil.which("dot") is None:
        return False
    return True

dot_source = generate_mindmap_dot(content)

if dot_source:
    print("\n✅ Generated DOT Source.")
    
    if check_graphviz():
        try:
            print("⏳ Rendering Mind Map...")
            dot = graphviz.Source(dot_source)
            # Verify we can actually render (catches ExecutableNotFound before display)
            dot.pipe(format='svg')
            display(dot)
        except Exception as e:
            print(f"❌ Visualization Error: {e}")
            print("\nHere is the raw DOT code (paste into http://magjac.com/graphviz-visual-editor/):")
            print(dot_source)
    else:
        print("⚠️  Graphviz binary not found. Cannot render image locally.")
        print("    To fix: Install Graphviz (https://graphviz.org/download/) and restart kernel.")
        print("    Windows Command: `winget install Graphviz.Graphviz`")
        print("\nHere is the raw DOT code (you can paste this into http://magjac.com/graphviz-visual-editor/):")
        print(dot_source)
else:
    print("❌ Failed to generate mind map.")

⏳ Generatng mind map structure...

✅ Generated DOT Source.
⚠️  Graphviz binary not found. Cannot render image locally.
    To fix: Install Graphviz (https://graphviz.org/download/) and restart kernel.
    Windows Command: `winget install Graphviz.Graphviz`

Here is the raw DOT code (you can paste this into http://magjac.com/graphviz-visual-editor/):
digraph MindMap {
    node [shape=box, style=filled, fillcolor=lightblue]
    AI_definition [label="Artificial Intelligence (AI)"]
    is_aofInterdisciplinaryScience [label="Wide-ranging branch of computer science", shape=box, style=filled, fillcolor=lightgrey]
    focusesOn[label="Building smart machines with human intelligence capabilities", shape=box, style=filled, fillcolor=lightgreen]
    advancesInSector [label="Creating paradigm shift in tech industry sectors through machine learning and deep learning", shape=box, style=filled, fillcolor=lightyellow]
    
    AI_definition -> is_aOfInterdisciplinaryScience [label="is a wide-ranging b